Generate expert dataset from csv file

In [ ]:
import pandas as pd
from tianshou.data import ReplayBuffer, Batch
import numpy as np

# Load the CSV file
df = pd.read_csv('pendulum_data_new.csv')

# Initialize the Replay Buffer
buffer_size = len(df)
expert_buffer = ReplayBuffer(buffer_size)

# Populate the Replay Buffer
for i in range(buffer_size):
    obs = np.array(df.loc[i, 'observations'][1:-1].split(', '), dtype=float)
    act = df.loc[i, 'actions']
    rew = df.loc[i, 'rewards']
    done = df.loc[i, 'terminals']
    next_obs = np.array(df.loc[i, 'next_observations'][1:-1].split(', '), dtype=float)
    
    # Assuming no truncation information is available, use the done flag for terminated
    # and False for truncated (change this based on your specific case)
    terminated = done
    truncated = False

    expert_buffer.add(
        Batch(
            obs=obs,
            act=act,
            rew=rew,
            done=done,
            obs_next=next_obs,
            terminated=terminated,
            truncated=truncated
        )
    )
print("Data loaded into Tianshou Replay Buffer.")

In [ ]:
expert_buffer._meta

Read expert dataset from saved pickle file

In [ ]:
import pickle
# from tianshou.data import VectorReplayBuffer  # Uncomment if needed

# Path to your pickle file containing the VectorReplayBuffer
pickle_file_path = 'expert_SAC_Pendulum-v1.pkl'

# Load the buffer from the pickle file
with open(pickle_file_path, 'rb') as file:
    buffer = pickle.load(file)

# Print the buffer metadata
print(dir(buffer)) 

In [ ]:
# Using __len__ to get the current size of the buffer
buffer_size = len(buffer)
print(f"Current Size of the Buffer: {buffer_size}")

# Using maxsize to get the maximum capacity of the buffer
buffer_capacity = buffer.maxsize
print(f"Maximum Capacity of the Buffer: {buffer_capacity}")

# Additional metadata
buffer_num = buffer.buffer_num
print(f"Number of Buffers: {buffer_num}")

stack_num = buffer.stack_num
print(f"Number of Stacked Frames: {stack_num}")

# Access the first buffer (if it's a VectorReplayBuffer)
single_buffer = buffer.buffers[0]

# Check if the buffer is not empty
if len(single_buffer) > 0:
    # Accessing the first transition
    # This depends on how the data is structured in your buffer
    # For example, if transitions are stored in a list-like structure
    first_transition = single_buffer[0]

    # Now, print the contents of the first transition
    print("First Transition Contents:")
    for key, value in first_transition.items():
        print(f"{key}: {value}")
else:
    print("The buffer is empty.")

Access the expert dataset in a single buffer

In [ ]:
# Access a specific buffer from the VectorReplayBuffer
specific_buffer = buffer.buffers[0]

# Now you can print this buffer or inspect its contents
print("Specific Buffer:", specific_buffer)

# To inspect details of this buffer, you can use dir() or access its attributes
# For example, printing the size of this specific buffer
print("Size of the Specific Buffer:", len(specific_buffer))